In [51]:
from minsearch import Index

In [19]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [61]:
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [66]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [67]:
q = 'can I enroll even after the course has been started?'

In [68]:
index.fit(documents)

In [76]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [78]:
results

[{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [72]:
from openai import OpenAI

In [73]:
client = OpenAI()

In [74]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user','content':q}]
)
response

ChatCompletion(id='chatcmpl-BfqO4fHTbP1d3VyBnyozVBUApNNJo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can enroll in a course after it has started depends on several factors such as the institution's policies, the course format, and the instructor's discretion. Here are a few steps you can take:\n\n1. **Check the Enrollment Policy**: Review the institution’s policies regarding late enrollment. Some courses may have specific deadlines while others might offer a more flexible enrollment period.\n\n2. **Contact the Instructor or Department**: If you’re interested in joining a course that’s already started, reach out to the instructor or the academic department. They might grant you permission to enroll late, especially if you have a valid reason for doing so.\n\n3. **Online Courses**: Many online or self-paced courses offer more flexibility, allowing students to join at any time. However, check the specific policies of 

In [82]:
context = ''
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [84]:
prompt_template="""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
    Use only the facts from the CONTEXT when answering the QUESTION
    If the CONTEXT doesn't contain the answer, output NONE
    Question: {question}

    CONTEXT:
    {context}
"""

In [86]:
promt = prompt_template.format(question=q, context=context).strip()

In [87]:
print(promt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
    Use only the facts from the CONTEXT when answering the QUESTION
    If the CONTEXT doesn't contain the answer, output NONE
    Question: can I enroll even after the course has been started?

    CONTEXT:
    section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything fo

In [88]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user','content': promt}]
)

In [89]:
response.choices[0].message.content

'Yes, you can enroll even after the course has started. You are still eligible to submit the homework and participate in the course activities. However, be mindful of the deadlines for turning in final projects to avoid last-minute issues.'

In [136]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [137]:
def build_prompt(query, search_results):
    prompt_template="""
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
        Use only the facts from the CONTEXT when answering the QUESTION
        If the CONTEXT doesn't contain the answer, output NONE
        Question: {question}
    
        CONTEXT:
        {context}
    """.strip()
    
    context = ''
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [138]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=[{'role':'user','content': prompt}]
    )
    return response.choices[0].message.content

In [139]:
query = 'how do I run kafka?'
search_results = search(query)
prompt = build_prompt(query, search_results)

In [140]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
        Use only the facts from the CONTEXT when answering the QUESTION
        If the CONTEXT doesn't contain the answer, output NONE
        Question: how do I run kafka?

        CONTEXT:
        section: Module 6: streaming with kafka
question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

section: Module 6: streaming with kafka
question: Module “kafka” not found when trying to run producer.py
answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual env):
source env/bin/activate

In [135]:
answer

'To run Kafka components (like producer, consumer, or kstreams) in the terminal, go to your project directory and use:\n\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nFor running Python Kafka producers, it is recommended to create and use a virtual environment, install dependencies with pip install -r ../requirements.txt, activate the virtual environment, and then run your Python files (e.g., producer.py) inside it. Also, make sure the Kafka broker Docker container is running (you can check with docker ps and start with docker compose up -d if needed).\n\nIf you have permission issues with build.sh on Python examples, run chmod +x build.sh in the same directory before using it.'

In [141]:
query = 'how do I run kafka?'
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [142]:
rag(query)

'To run Kafka-related Java programs (like producer/consumer/kstreams/etc) in the project directory, use the following command in your terminal:\n\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nIf you are running Python Kafka producers/consumers, you should:\n1. Create and activate a virtual environment:\n   python -m venv env\n   source env/bin/activate\n2. Install dependencies:\n   pip install -r ../requirements.txt\n3. Run the Python Kafka files within this environment.\n\nMake sure Docker images are up and running before running the Python file.'

In [145]:
from elasticsearch import Elasticsearch

In [146]:
es_client = Elasticsearch("http://localhost:9200")

In [147]:
es_client.info()

ObjectApiResponse({'name': '5150f25d3717', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'WRu0jZ7FSdm9AnpnWn7VKQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [148]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [149]:
from tqdm.auto import tqdm

In [150]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [165]:
query = 'I just discovered the course. Can I still join it?'

In [166]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [167]:
result_docs = elastic_search(query)

In [168]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [169]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [170]:
rag(query)

"Yes, you can still join the course even after the start date. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."